In [1]:
# 0.52112
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)

from scipy import stats
from scipy.stats import norm, skew #for some statistics

pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
#Save the 'Id' column
train_ID = train['id']
test_ID = test['id']

#Now drop the  'Id' colum since it's unnecessary for  the prediction process.
train.drop("id", axis = 1, inplace = True)
test.drop("id", axis = 1, inplace = True)

In [4]:
train.describe()

,l1_ratio,alpha,max_iter,random_state,n_jobs,n_samples,n_features,n_classes,n_clusters_per_class,n_informative,flip_y,scale,time
count,400.000,400.000,400.000,400.000,400.000,400.000,400.000,400.000,400.000,400.000,400.000,400.000,400.000
mean,0.503,0.004,555.510,489.210,2.810,756.210,766.145,5.855,3.458,8.675,0.049,50.339,3.202
std,0.232,0.004,265.000,287.456,2.951,373.326,375.214,2.493,1.101,1.522,0.030,28.993,5.266
min,0.100,0.000,103.000,0.000,-1.000,100.000,110.000,2.000,2.000,5.000,0.000,1.528,0.075
25%,0.300,0.000,333.250,232.250,1.000,451.500,439.750,4.000,3.000,8.000,0.021,25.230,0.504
50%,0.492,0.001,532.000,499.000,2.000,748.000,774.000,6.000,3.000,9.000,0.047,51.630,1.405
75%,0.704,0.010,787.250,728.000,4.000,1065.750,1088.500,8.000,4.000,10.000,0.075,74.948,3.629
max,0.895,0.010,998.000,1000.000,8.000,1397.000,1397.000,10.000,5.000,12.000,0.100,99.748,41.100


In [5]:
test.describe()

,l1_ratio,alpha,max_iter,random_state,n_jobs,n_samples,n_features,n_classes,n_clusters_per_class,n_informative,flip_y,scale
count,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000
mean,0.513,0.003,510.740,487.260,5.030,1055.130,1086.700,6.130,3.370,8.580,0.052,50.018
std,0.227,0.004,251.590,296.587,5.595,579.634,525.249,2.501,1.134,1.742,0.029,25.160
min,0.109,0.000,104.000,4.000,-1.000,116.000,148.000,2.000,2.000,5.000,0.000,2.153
25%,0.348,0.000,291.000,237.500,1.000,524.250,655.250,4.000,2.000,7.000,0.031,27.698
50%,0.524,0.001,486.000,478.000,4.000,1028.500,1117.500,6.000,3.000,8.000,0.052,55.086
75%,0.706,0.010,728.500,765.750,8.000,1589.500,1501.250,8.000,4.000,10.000,0.076,68.791
max,0.900,0.010,983.000,977.000,16.000,1993.000,1993.000,10.000,5.000,12.000,0.100,98.798


In [4]:
ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train.time.values
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop(['time'], axis=1, inplace=True)

In [5]:
print("all_data size is : {}".format(all_data.shape))

all_data size is : (500, 13)


In [6]:
all_data.drop(['l1_ratio', 'alpha', 'random_state', 'n_clusters_per_class', 
               'flip_y','scale'], axis=1, inplace=True)

In [7]:
for i in range(len(all_data)):
    if all_data['n_jobs'][i] == -1: 
        all_data['n_jobs'][i] = 16

In [8]:
from sklearn.preprocessing import LabelEncoder
cols = ('penalty', 'n_classes',  'n_informative', 'n_jobs')
# process columns, apply LabelEncoder to categorical features
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(all_data[c].values)) 
    all_data[c] = lbl.transform(list(all_data[c].values))

In [9]:
# shape        
print('Shape all_data: {}'.format(all_data.shape))

Shape all_data: (500, 7)


In [10]:
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)


Skew in numerical features: 



,Skew
n_samples,0.395
n_features,0.202
n_informative,0.117
max_iter,0.045
n_classes,0.034
penalty,0.028
n_jobs,0.014


In [11]:
skewness = skewness[abs(skewness) > 0.75]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    #all_data[feat] += 1
    all_data[feat] = boxcox1p(all_data[feat], lam)

There are 7 skewed numerical features to Box Cox transform


In [12]:
all_data = pd.get_dummies(all_data)
print(all_data.shape)

(500, 7)


In [13]:
train = all_data[:ntrain]
test = all_data[ntrain:]

In [14]:
#We use the numpy fuction log1p which  applies log(1+x) to all elements of the column
y_train = np.log1p(y_train)

In [15]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [16]:
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [17]:
all_data.columns

Index(['max_iter', 'n_classes', 'n_features', 'n_informative', 'n_jobs',
       'n_samples', 'penalty'],
      dtype='object')

In [20]:
# train.drop([''], axis=1, inplace=True)
# test.drop([''], axis=1, inplace=True)

In [18]:
train.shape

(400, 7)

In [19]:
mlp1 = make_pipeline(RobustScaler(), MLPRegressor(hidden_layer_sizes=(100, 50, 30, 30), activation='tanh', 
                                                  alpha=0.11, 
                                                  solver = 'lbfgs',
#                                                   learning_rate='adaptive', 
                                                  max_iter=20000, tol=1e-10))
# mlp = make_pipeline(RobustScaler(), MLPRegressor(hidden_layer_sizes=(200, 100, 80, 40, 30), activation='tanh', 
#                                                  alpha=0.05, max_iter=2000, tol=1e-10))

In [20]:
mlp2 = make_pipeline(RobustScaler(), MLPRegressor(hidden_layer_sizes=(100, 50, 30, 30), activation='tanh', 
                                                  alpha=0.115, 
                                                  solver = 'lbfgs',
#                                                   learning_rate='adaptive', 
                                                  max_iter=20000, tol=1e-10))

In [21]:
mlp3 = make_pipeline(RobustScaler(), MLPRegressor(hidden_layer_sizes=(100, 50, 30, 30), activation='tanh', 
                                                  alpha=0.11, 
                                                  solver = 'lbfgs',
#                                                   learning_rate='adaptive', 
                                                  max_iter=20000, tol=1e-10))

In [22]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, min_samples_split=10, 
                                   loss='huber', random_state =5,
                                  tol=1e-10)

In [23]:
model_xgb = xgb.XGBRegressor()

In [24]:
model_lgb = lgb.LGBMRegressor()

In [25]:
score = rmsle_cv(mlp1)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Lasso score: 0.0729 (0.0075)



In [26]:
score = rmsle_cv(mlp2)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Lasso score: 0.0746 (0.0080)



In [27]:
score = rmsle_cv(mlp3)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Lasso score: 0.0745 (0.0096)



In [28]:
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Gradient Boosting score: 0.2340 (0.0378)



In [29]:
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Xgboost score: 0.2271 (0.0417)



In [30]:
score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

LGBM score: 0.2203 (0.0391)



In [39]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)  

In [40]:
averaged_models = AveragingModels(models = (mlp1, mlp2, mlp3))

score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

 Averaged base models score: 0.1705 (0.0247)



In [40]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [41]:
stacked_averaged_models = StackingAveragedModels(base_models = (mlp1, mlp2, mlp3),
                                                 meta_model = model_lgb)

score = rmsle_cv(stacked_averaged_models)
print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

Stacking Averaged models score: 0.1309 (0.0408)


In [31]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [42]:
stacked_averaged_models.fit(train.values, y_train)
stacked_train_pred = stacked_averaged_models.predict(train.values)
stacked_pred = np.expm1(stacked_averaged_models.predict(test.values))
print(rmsle(y_train, stacked_train_pred))

0.09799704652961656


In [32]:
model_xgb.fit(train, y_train)
xgb_train_pred = model_xgb.predict(train)
xgb_pred = np.expm1(model_xgb.predict(test))
print(rmsle(y_train, xgb_train_pred))

0.09383280053978237


In [33]:
model_lgb.fit(train, y_train)
lgb_train_pred = model_lgb.predict(train)
lgb_pred = np.expm1(model_lgb.predict(test.values))
print(rmsle(y_train, lgb_train_pred))

0.06716992176062296


In [32]:
mlp1.fit(train, y_train)
mlp1_train_pred = mlp1.predict(train)
mlp1_pred = np.expm1(mlp1.predict(test.values))
print(rmsle(y_train, mlp1_train_pred))

0.046845982186925345


In [33]:
mlp2.fit(train, y_train)
mlp2_train_pred = mlp2.predict(train)
mlp2_pred = np.expm1(mlp2.predict(test.values))
print(rmsle(y_train, mlp2_train_pred))

0.04984774414760795


In [34]:
mlp3.fit(train, y_train)
mlp3_train_pred = mlp3.predict(train)
mlp3_pred = np.expm1(mlp3.predict(test.values))
print(rmsle(y_train, mlp3_train_pred))

0.0490883842999656


In [35]:
print('RMSLE score on train data:')
# print(rmsle(y_train,stacked_train_pred*0.90 + xgb_train_pred*0.05 + lgb_train_pred*0.05))
print(rmsle(y_train,mlp1_train_pred*0.40 + mlp2_train_pred*0.30 + mlp3_train_pred*0.30))

RMSLE score on train data:
0.04729598867740608


In [36]:
# ensemble = stacked_pred*0.90 + xgb_pred*0.05 + lgb_pred*0.05
# ensemble = mlp1_pred*0.5 + mlp2_pred*0.5
# ensemble = mlp2_pred
ensemble = mlp1_pred*0.4 + mlp2_pred*0.3 + mlp3_pred*0.3

In [38]:
sub = pd.DataFrame()
sub['id'] = test_ID
sub['time'] = ensemble
sub.to_csv('time.csv',index=False)